In [70]:
from pymatgen.analysis.wulff import WulffShape

# Import the necessary tools to generate surfaces
from pymatgen.core.surface import Lattice, SlabGenerator, Structure, generate_all_slabs

from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.cif import CifWriter

from ase.visualize import view

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.ase import AseAtomsAdaptor

In [2]:
lattice = Lattice.cubic(3.508)
Ni = Structure(
    lattice,
    ["Ni", "Ni", "Ni", "Ni"],
    [[0, 0, 0], [0, 0.5, 0], [0.5, 0, 0], [0, 0, 0.5]],
)

In [3]:
slabgen = SlabGenerator(Ni, (1, 1, 1), 10, 10)

all_slabs = slabgen.get_slabs()

In [74]:
surface_energies_Ni = surface_energies_Ni = {
    (3, 2, 0): 2.3869,
    (1, 1, 0): 2.2862,
    (3, 1, 0): 2.3964,
    (2, 1, 0): 2.3969,
    (3, 3, 2): 2.0944,
    (1, 0, 0): 2.2084,
    (2, 1, 1): 2.2353,
    (3, 2, 2): 2.1242,
    (3, 2, 1): 2.3183,
    (2, 2, 1): 2.1732,
    (3, 3, 1): 2.2288,
    (3, 1, 1): 2.3039,
    (1, 1, 1): 1.9235,
}
miller_list = surface_energies_Ni.keys()
e_surf_list = surface_energies_Ni.values()

# We can now construct a Wulff shape with an accuracy up to a max Miller index of 3
wulffshape = WulffShape(Ni.lattice, miller_list, e_surf_list)

# Let's get some useful information from our wulffshape object
print(
    "shape factor: %.3f, anisotropy: \
%.3f, weighted surface energy: %.3f J/m^2"
    % (
        wulffshape.shape_factor,
        wulffshape.anisotropy,
        wulffshape.weighted_surface_energy,
    )
)


# If we want to see what our Wulff shape looks like
#wulffshape.show()

shape factor: 5.178, anisotropy: 0.070, weighted surface energy: 2.035 J/m^2


In [23]:
wulffshape.facets[0].points

[[array([2.2084, 2.2084, 2.2084]),
  array([ 2.2084,  2.2084, -2.2084]),
  array([ 2.2084, -2.2084, -2.2084])],
 [array([2.2084, 2.2084, 2.2084]),
  array([ 2.2084, -2.2084,  2.2084]),
  array([ 2.2084, -2.2084, -2.2084])]]

In [19]:
wulffshape.get_line_in_facet(wulffshape.facets[1])

[[-2.2084, -2.2084, 2.2084],
 [-2.2084, -2.2084, -2.2084],
 [-2.2084, -2.2084, -2.2084],
 [-2.2084, 2.2084, -2.2084],
 [-2.2084, 2.2084, -2.2084],
 [-2.2084, 2.2084, 2.2084],
 [-2.2084, 2.2084, 2.2084],
 [-2.2084, -2.2084, 2.2084]]

In [75]:
wulffshape.wulff_convex.points

array([[-2.2084    , -0.94283134, -0.07314052],
       [ 2.2084    , -0.94283134, -0.07314052],
       [-2.2084    , -0.94283134,  0.07314052],
       [-0.94283134, -2.2084    , -0.07314052],
       [-0.07314052, -2.2084    , -0.94283134],
       [-0.94283134, -2.2084    ,  0.07314052],
       [-0.07314052, -2.2084    ,  0.94283134],
       [ 0.94283134, -2.2084    , -0.07314052],
       [ 0.07314052, -2.2084    , -0.94283134],
       [ 0.94283134, -2.2084    ,  0.07314052],
       [ 0.07314052, -2.2084    ,  0.94283134],
       [-2.17468091, -1.01026951, -0.10685961],
       [-0.94283134, -0.07314052, -2.2084    ],
       [-0.07314052, -0.94283134, -2.2084    ],
       [-1.10671876, -0.01558883, -2.12645629],
       [-2.2084    , -0.07314052, -0.94283134],
       [-2.2084    ,  0.07314052, -0.94283134],
       [-2.12645629, -0.01558883, -1.10671876],
       [-2.17468091, -0.10685961, -1.01026951],
       [-2.15331968, -0.09617899, -1.05299198],
       [ 2.17468091, -1.01026951, -0.106

In [42]:
poly = []
for i in range(6):
    #.extend(wulffshape.get_line_in_facet(wulffshape.facets[i]))
    poly.extend(wulffshape.facets[i].points)

In [51]:
from scipy.spatial import Delaunay
import numpy as np
poly = np.array([[0,0.5,0],[1.8,1,1],[2,2,2.1],[3,3.4,3]])
poly= np.array(wulffshape.get_line_in_facet(wulffshape.facets[1]))
test = []
for i in range(50000):
    point = np.array([i,i,i])
    test.append(Delaunay(wulffshape.wulff_convex.points).find_simplex(point) >= 0)

In [117]:
lattice =  np.array(  [[0.000000000000E+00 ,  0.210850000000E+01 ,  0.210850000000E+01],
   [0.210850000000E+01 ,  0.000000000000E+00  , 0.210850000000E+01],
   [0.210850000000E+01 ,  0.210850000000E+01 ,  0.000000000000E+00]])
structure = Structure(lattice,['Mg','O'],[[0.,0.,0.],[0.5,0.5,0.5]])
structure = SpacegroupAnalyzer(structure).get_conventional_standard_structure()

supercell_matrix = np.identity(3)*10
structure.make_supercell(supercell_matrix)
center_of_mass = np.average(structure.cart_coords,axis=0)
structure.translate_sites(np.arange(structure.num_sites),-center_of_mass,frac_coords=False,to_unit_cell= False)
structure.num_sites

8000

In [119]:
shape = Delaunay(wulffshape.wulff_convex.points*10)
sites_to_remove = []
for i,coord in enumerate(structure.cart_coords):
    if (shape.find_simplex(coord) >= 0) == False:
        sites_to_remove.append(i)
structure.remove_sites(sites_to_remove)
structure.

In [120]:
view(AseAtomsAdaptor().get_atoms(structure))

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/test_env...>